<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
import torchaudio
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
# train_dataset = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)

print(valid_dataset)

# # Example: Load the first audio file in the dataset
first_sample = valid_dataset[0]
dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

labels = valid_dataset.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

def edit_exam(examples):
    for i, label in enumerate(examples):
        label2id[label] = str(i)
        id2label[str(i)] = label
        # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
        for i, x in enumerate(examples['file']):
            a = examples['label'][i]
            if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
            elif id2label[str(examples['label'][i])] == '_silence_':
                examples['label'][i] = dict_label['silence']
    return examples



valid_dataset = valid_dataset.map(edit_exam, batched=True)
df = valid_dataset.to_pandas()
# Giả sử cột label trong dataset có tên là 'label'
label_counts = df['label'].value_counts()

# Hiển thị kết quả
print(label_counts)

# Hiển thị đồ thị
label_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Số lượng')
plt.title('Số lượng mỗi class trong cột label')
plt.show()
# valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', 'silence']
# first_sample

In [ ]:
from transformers import AutoFeatureExtractor
from datasets import load_dataset, Audio
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer
import numpy as np
import evaluate
accuracy = evaluate.load("accuracy")


import pandas as pd
import matplotlib.pyplot as plt

dict_label = {'yes':0,
              'no':1,
              'up':2,
              'down':3,
              'left':4,
              'right':5,
              'on':6,
              'off':7,
              'stop':8,
              'go':9,
              'unknown':10,
              'silence':11}

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16_000, truncation=True
    )
    return inputs




def edit_label(examples):
    # valid_dataset.features['label'].names = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go', 'unknown', '_silence_']
    for i, x in enumerate(examples['file']):
        a = examples['label'][i]
        if examples['is_unknown'][i]: examples['label'][i] = dict_label['unknown']
        elif id2label[str(examples['label'][i])] == '_silence_':
            examples['label'][i] = dict_label['silence']
    return examples

feature_extractor = AutoFeatureExtractor.from_pretrained('./model_1')
train_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='train', trust_remote_code=True)
valid_dataset_ = load_dataset("google/speech_commands", 'v0.01', split='validation', trust_remote_code=True)


labels = valid_dataset_.features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label


train_dataset_ = train_dataset_.map(edit_label, batched=True)
valid_dataset_ = valid_dataset_.map(edit_label, batched=True)


df = train_dataset_.to_pandas()
# Giả sử cột label trong dataset có tên là 'label'
label_counts = df['label'].value_counts()

# Hiển thị kết quả
print(label_counts)

# Hiển thị đồ thị
label_counts.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Số lượng')
plt.title('Số lượng mỗi class trong cột label')
plt.show()

train_dataset_ = train_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])
valid_dataset_ = valid_dataset_.remove_columns(["file", "is_unknown", "speaker_id", "utterance_id"])

labels = dict_label.keys()
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label
print(label2id)



train_data = train_dataset_.cast_column("audio", Audio(sampling_rate=16_000))
valid_data = valid_dataset_.cast_column("audio", Audio(sampling_rate=16_000))

encoded_data_train = train_data.map(preprocess_function, remove_columns="audio", batched=True)
encoded_data_validation = valid_data.map(preprocess_function, remove_columns="audio", batched=True)
print(encoded_data_train)

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "./model_1", num_labels=num_labels, label2id=label2id, id2label=id2label
)
training_args = TrainingArguments(
    output_dir="my_awesome_mind_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=10,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_data_train.with_format("torch"),
    eval_dataset=encoded_data_validation.with_format("torch"),
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()